In [13]:
import numpy as np
import pandas as pd
import os
import gc
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import seaborn 
import matplotlib.pyplot as pl
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

In [14]:
path = '../../data/train.csv'

In [15]:
# chunks = pd.read_csv(path, low_memory=True, chunksize=1000000, sep=',')

# df=pd.DataFrame()
# %time train=pd.concat(chunk for chunk in chunks[:2])

In [16]:
train = pd.read_csv(path, low_memory=True, nrows=500000)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
train.shape

(500000, 83)

In [18]:
train = train.fillna(0)

In [19]:
sel_cols = [c for c in train.columns if c not in ['MachineIdentifier',
                                                      'HasDetections'
                                                     ]]

In [20]:
X_train = train.loc[:, sel_cols]
y_train = train.loc[:,'HasDetections']
del train
gc.collect()

18

In [21]:
cat_cols = list()
n = 0
for c, t in zip(sel_cols, X_train.dtypes):
    if t == 'object':
        cat_cols.append(n)
    n+=1

In [22]:
train_ids = X_train.index
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, y_train)

5

In [29]:
model_cb = CatBoostClassifier(
    depth=9,
    iterations=100,
    eval_metric='AUC',
    random_seed=42,
    logging_level='Verbose',
    allow_writing_files=False,
    metric_period=5,
    early_stopping_rounds=20,
    learning_rate=0.1,
    bagging_temperature=0.9
    
)

# PARAMETROS:
# iterations=None,
#                          learning_rate=None,
#                          depth=None,
#                          l2_leaf_reg=None,
#                          model_size_reg=None,
#                          rsm=None,
#                          loss_function='Logloss',
#                          border_count=None,
#                          feature_border_type=None,
#                          old_permutation_block_size=None,
#                          od_pval=None,
#                          od_wait=None,
#                          od_type=None,
#                          nan_mode=None,
#                          counter_calc_method=None,
#                          leaf_estimation_iterations=None,
#                          leaf_estimation_method=None,
#                          thread_count=None,
#                          random_seed=None,
#                          use_best_model=None,
#                          verbose=None,
#                          logging_level=None,
#                          metric_period=None,
#                          ctr_leaf_count_limit=None,
#                          store_all_simple_ctr=None,
#                          max_ctr_complexity=None,
#                          has_time=None,
#                          allow_const_label=None,
#                          classes_count=None,
#                          class_weights=None,
#                          one_hot_max_size=None,
#                          random_strength=None,
#                          name=None,
#                          ignored_features=None,
#                          train_dir=None,
#                          custom_loss=None,
#                          custom_metric=None,
#                          eval_metric=None,
#                          bagging_temperature=None,
#                          save_snapshot=None,
#                          snapshot_file=None,
#                          snapshot_interval=None,
#                          fold_len_multiplier=None,
#                          used_ram_limit=None,
#                          gpu_ram_part=None,
#                          allow_writing_files=None,
#                          final_ctr_computation_mode=None,
#                          approx_on_full_history=None,
#                          boosting_type=None,
#                          simple_ctr=None,
#                          combinations_ctr=None,
#                          per_feature_ctr=None,
#                          task_type=None,
#                          device_config=None,
#                          devices=None,
#                          bootstrap_type=None,
#                          subsample=None,
#                          max_depth=None,
#                          n_estimators=None,
#                          num_boost_round=None,
#                          num_trees=None,
#                          colsample_bylevel=None,
#                          random_state=None,
#                          reg_lambda=None,
#                          objective=None,
#                          eta=None,
#                          max_bin=None,
#                          scale_pos_weight=None,
#                          gpu_cat_features_storage=None,
#                          data_partition=None
#                          metadata=None, 
#                          early_stopping_rounds=None,
#                          cat_features=None)

In [30]:
counter = 0
for train_index, test_index in skf.split(train_ids, y_train):
    
    print('Fold {}\n'.format(counter + 1))
    
    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model_cb.fit(X_fit,
                 y_fit,
                 cat_features=cat_cols,
                 eval_set=(X_val, y_val),
#                  plot=True
                )
                  
    del X_fit, X_val, y_fit, y_val, train_index, test_index
    gc.collect()
    
    counter += 1

Fold 1



0:	test: 0.6711455	best: 0.6711455 (0)	total: 1.14s	remaining: 1m 53s
5:	test: 0.6847151	best: 0.6847151 (5)	total: 6.54s	remaining: 1m 42s
10:	test: 0.6881688	best: 0.6881688 (10)	total: 12.2s	remaining: 1m 38s
15:	test: 0.6910411	best: 0.6910411 (15)	total: 18s	remaining: 1m 34s
20:	test: 0.6935162	best: 0.6935162 (20)	total: 23.7s	remaining: 1m 29s
25:	test: 0.6956813	best: 0.6956813 (25)	total: 29.2s	remaining: 1m 23s
30:	test: 0.6973621	best: 0.6973621 (30)	total: 35s	remaining: 1m 17s
35:	test: 0.6982753	best: 0.6982753 (35)	total: 40.9s	remaining: 1m 12s
40:	test: 0.6997018	best: 0.6997018 (40)	total: 46.7s	remaining: 1m 7s
45:	test: 0.7007818	best: 0.7007818 (45)	total: 52.7s	remaining: 1m 1s
50:	test: 0.7016437	best: 0.7016437 (50)	total: 58.4s	remaining: 56.1s
55:	test: 0.7024359	best: 0.7024359 (55)	total: 1m 4s	remaining: 50.4s
60:	test: 0.7035671	best: 0.7035671 (60)	total: 1m 10s	remaining: 44.9s
65:	test: 0.7043224	best: 0.7043224 (65)	total: 1m 16s	remaining: 39.2s
70:	

0:	test: 0.6660307	best: 0.6660307 (0)	total: 1.25s	remaining: 2m 3s
5:	test: 0.6824893	best: 0.6824893 (5)	total: 7.12s	remaining: 1m 51s
10:	test: 0.6883729	best: 0.6883729 (10)	total: 13.1s	remaining: 1m 45s
15:	test: 0.6911804	best: 0.6911804 (15)	total: 19.1s	remaining: 1m 40s
20:	test: 0.6933084	best: 0.6933084 (20)	total: 25.1s	remaining: 1m 34s


KeyboardInterrupt: 